<h1> Webscraping dos XML Files </h1>

In [7]:
# Imports gerais
import time
import os
import zipfile
from datetime import datetime

In [8]:
# Imports da selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

In [9]:
def configure_chrome_driver(download_dir: str) -> tuple:
    """
    Configura o Chrome Driver para download automático e execução em background.
    """
    # Configurações do Chrome Driver para download automático
    chrome_options = webdriver.ChromeOptions()
    chrome_prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }

    # Configurações do Chrome Driver para execução em background
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    chrome_options.add_experimental_option("prefs", chrome_prefs)

    service = ChromeService(ChromeDriverManager().install())

    return service, chrome_options

In [10]:
def webscraping_pregao(
        url: str,
        download_dir: str,
        service: any,
        chrome_options: any) -> str:
    """
    Faz todo o processo de download do arquivo .zip com o histórico de negociações,
    retornando o caminho do arquivo .zip baixado.
    """
    # Inicia o WebDriver e acessa a URL
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)

    try:
        # Espera até que o checkbox esteja disponível e clica nele
        checkbox = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, "8AA8D0975C8A570C015C8E080B9D3700"))
        )
        checkbox.click()

        # Espera até que o botão de download esteja disponível e clica nele
        download_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, "botao-download"))
        )
        download_button.click()

        # Monitora a pasta de downloads para verificar se o arquivo foi baixado
        download_complete = False
        while not download_complete:
            # Espera 5 segundos e verifica novamente
            time.sleep(5)
            for filename in os.listdir(download_dir):
                if filename.endswith(".zip"):
                    zip_file_path = os.path.join(download_dir, filename)
                    download_complete = True
                    break

    finally:
        # Encerra o WebDriver
        driver.quit()
    
    return zip_file_path

In [11]:
def extract_all_files(zip_file_path: str, extract_to: str) -> None:
    """
    Extrai todos os arquivos de um arquivo zip, incluindo aninhados.
    """
    # Verifica a integridade do arquivo zip
    if not zipfile.is_zipfile(zip_file_path):
        raise ValueError(f"O arquivo {zip_file_path} não é um arquivo zip válido.")

    # Extrai todos os arquivos do arquivo zip
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

        # Verifica e extrai arquivos zip aninhados (usa recursão)
        for file_name in zip_ref.namelist():
            extracted_path = os.path.join(extract_to, file_name)
            if zipfile.is_zipfile(extracted_path):
                extract_all_files(extracted_path, extract_to)
                # Remove o arquivo zip aninhado após a extração
                os.remove(extracted_path)  

In [12]:
def get_pesquisa_pregao() -> None:
    """
    Função principal do script.
    
    Realiza o download do arquivo .zip contendo o histórico de negociações
    da B3 por pregão e extrai os arquivos XMLs contidos no .zip de forma
    versionada e organizada, separando ZIPs e XMLs. Por fim, gera um
    arquivo .parquet com todas as informações do XML.
    """
    # Declarando URL da página do histórico de negociações por pregão
    url = (
        "https://www.b3.com.br/pt_br/market-data-e-indices/"
        "servicos-de-dados/market-data/historico/boletins-diarios/"
        "pesquisa-por-pregao/pesquisa-por-pregao/"
    )
    
    # Criando o diretório de download com a data atual
    today = datetime.today().strftime('%Y-%m-%d')
    base_dir = os.path.join(os.getcwd(), 'data', today)
    download_dir = os.path.join(base_dir, 'zip_files')
    xml_dir = os.path.join(base_dir, 'xml_files')

    os.makedirs(download_dir, exist_ok=True)
    os.makedirs(xml_dir, exist_ok=True)
    
    # Função para configurar o Chrome Driver
    service, chrome_options = configure_chrome_driver(download_dir)
    
    # Função para fazer o webscraping baixando o arquivo .zip
    zip_file_path = webscraping_pregao(url, download_dir, service, chrome_options)
    
    # Função para extrair os arquivos XML do .zip
    extract_all_files(zip_file_path, xml_dir)

get_pesquisa_pregao()